<a href="https://colab.research.google.com/github/kimjaehwankimjaehwan/python_/blob/main/basline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
# 데이터 경로
data_path = '/kaggle/input/bike-sharing-demand/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

In [ ]:
# 이상치 제거
train = train[train['weather'] != 4]

In [ ]:
all_data_temp = pd.concat([train, test])
all_data_temp

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data

In [ ]:
# 파생 피처(변수) 추가
from datetime import datetime

# 날짜 피처 생성
all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])
# 연도 피처 생성
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])
# 월 피처 생성
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])
# 시 피처 생성
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])
# 요일 피처 생성
all_data["weekday"] = all_data['date'].apply(lambda dateString : datetime.strptime(dateString,"%Y-%m-%d").weekday())

In [ ]:
all_data.head(2)

In [ ]:
# 필요 없는 피처 제거
drop_features = ['casual', 'registered', 'datetime', 'date', 'month', 'windspeed']

In [ ]:
all_data = all_data.drop(drop_features, axis=1)

In [ ]:
all_data.head(2)

In [ ]:
# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data[~pd.isnull(all_data['count'])]
X_test = all_data[pd.isnull(all_data['count'])]

# 타깃값 count 제거
X_train = X_train.drop(['count'], axis=1)
X_test = X_test.drop(['count'], axis=1)

y = train['count']

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
import numpy as np

def rmsle(y_true, y_pred, convertExp=True):
    # 지수변환
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)

    # 로그변환 후 결측값을 0으로 변환
    log_true = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))

    # RMSLE 계산
    output = np.sqrt(np.mean((log_true - log_pred)**2))
    return output

In [ ]:
# 모델 훈련
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()

In [ ]:
# 훈련 데이터로 모델을 훈련
log_y = np.log(y)  # 타깃값 로그변환
linear_reg_model.fit(X_train, log_y) # 모델 훈련

In [ ]:
# 모델 성능 검증
preds = linear_reg_model.predict(X_train)

print('선형 회귀의 RMSLE 값: {:.4f}'.format(rmsle(log_y, preds, True)))

In [ ]:
# 예측 및 결과 제출
linearreg_preds = linear_reg_model.predict(X_test)       # 테스트 데이터로 예측

submission['count'] = np.exp(linearreg_preds)            # 지수변환
submission.to_csv('submission.csv', index=False)         # 파일로 저장

#### 성능 개선 I : 릿지 회귀 모델
---

In [ ]:
# 모델 생성
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

ridge_model = Ridge()

In [ ]:
# 하이퍼 파라미터 값 목록
ridge_params = {'max_iter': [3000],
                'alpha': [0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400, 800, 900, 1000]}

# 교차 검증용 평가 함수(RMSEL 점수 계산)
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# 그리드서치(with 릿지) 객체 생성
gridsearch_ridge_model = GridSearchCV(estimator = ridge_model,   # 릿지 모델
                                      param_grid=ridge_params,   # 값 목록
                                      scoring=rmsle_scorer,      # 평가지표
                                      cv=5)                      # 교차 검증 분할 수

In [ ]:
# 그리드서치 수행
log_y = np.log(y)
gridsearch_ridge_model.fit(X_train, log_y)   # 훈련

In [ ]:
# 성능 검증
# 예측
preds = gridsearch_ridge_model.best_estimator_.predict(X_train)

# 평가
print('릿지 회귀 RMSLE 값 : {:.4f}'.format(rmsle(log_y, preds, True)))

### 성능 개선 II : 라쏘 회귀 모델
---

In [ ]:
# 하이퍼파라미터 최적화(모델 훈련)
from sklearn.linear_model import Lasso

# 모델 생성
lasso_model = Lasso()

# 하이퍼파라미터 값 목록
lasso_alpha = 1/np.array([0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400, 800, 900, 1000])
lasso_params = {'max_iter': [3000], 'alpha': lasso_alpha}

# 그리드서치(with 라쏘) 객체 생성
gridsearch_lasso_model = GridSearchCV(estimator=lasso_model,
                                     param_grid=lasso_params,
                                     scoring=rmsle_scorer,
                                     cv=5)

# 그리드서치 수행
log_y = np.log(y)
gridsearch_lasso_model.fit(X_train, log_y)

print('최적 하이퍼파라미터', gridsearch_lasso_model.best_params_)

In [ ]:
# 성능 검증
# 예측
greds = gridsearch_lasso_model.best_estimator_.predict(X_train)

# 평가
print('라쏘 회귀 RMSLE 값 {:.4f}'.format(rmsle(log_y, preds, True)))

### 성능 개선 III : 랜덤 포레스트 회귀 모델
---

In [ ]:
# 하이퍼파라미터 최적화(모델 훈련)
from sklearn.ensemble import RandomForestRegressor

# 모델 생성
randomforest_model = RandomForestRegressor()

# 그리드서치 객체 생성
rf_params = {'random_state': [42], 'n_estimators': [100, 120, 140]}
gridsearch_random_forest_model = GridSearchCV(estimator=randomforest_model,
                                              param_grid=rf_params,
                                              scoring=rmsle_scorer,
                                              cv=5)

# 그리드서치 수행
log_y = np.log(y)
gridsearch_random_forest_model.fit(X_train, log_y)

print('최적 하이퍼파라미터 :', gridsearch_random_forest_model.best_params_)

In [ ]:
# 예측 및 결과 제출
import seaborn as sns
import matplotlib.pyplot as plt

randomforest_preds = gridsearch_random_forest_model.best_estimator_.predict(X_test)

figure, axes = plt.subplots(ncols=2)
figure.set_size_inches(10, 4)

sns.histplot(y, bins=50, ax=axes[0])
axes[0].set_title('Train Data Distribution')
sns.histplot(np.exp(randomforest_preds), bins=50, ax=axes[1])
axes[1].set_title('Predicted Test Data Distribution')

In [ ]:
submission['count'] = np.exp(randomforest_preds)   # 지수변환
submission.to_csv('submission.csv', index=False)